# Deep Dive Into Markov Models/Chains

In [4]:
# load packages
library(expm)

Markov Chains are actually extremely intuitive. Formally, they are examples of Stochastic Processes, or random variables that evolve over time. The time can be measured as discrete or continuous. For simplicity, this lab will only cover Markov models in the context of **discrete time**. You can begin to visualize a Markov Chain as a random process bouncing between different states. Here is a basic but classic example of what a Markov chain can actually look like:

![](MC1.png)

It’s probably not hard to tell what’s going on here, even without thorough explanation. This Markov Chain is describing the random process of weather over multiple days. In this simple example, we’re assuming that it can either be sunny or rainy in a single day. If today is sunny, then tomorrow there is a .2 probability that it is rainy, and a .8 probability that it is sunny again (shown here by the arrows leaving from the ‘Sunny’ circle). If it’s rainy, then tomorrow there is a .3 probability that it it rainy again on the next day and a .7 probability that it is sunny on the next day. You can envision a particle bouncing around this chain, moving once each day according to the probabilities drawn here.

Makes sense, right? Let’s talk about some simple terminology. We’re going to call the circles - here we have one that says ‘Sunny’ and one that says ‘Rainy’ - **nodes** (also often called **states**), and the arrows connecting them **edges**. Each edge has a probability associated with it; as specified above, this is the probability of ‘following’ that edge if you start at that node.

Note that the probabilities of all the edges leaving out of a specific node must sum to $1$. For example, if it’s Sunny, we have a $.8$ probability and a $.2$ probability of it being Sunny and Rainy tomorrow, respectively, and $.8+.2=1$. This makes sense; if we start at a node, we have to go somewhere on the next step, so the probabilities of all of the edges leaving the nodes must add up to 1. If you had a node where the edge probabilities summed to $.9$, for example, this wouldn’t make much sense. Where would the particle move $10\%$ of the time?



The next step is to define the Transition Matrix, which we usually notate as  Q . This is simply the matrix that contains the probabilities of, you guessed it, transferring between nodes. In this case, the transition matrix looks like:

$Q$ = \begin{array}
{rr}
.8 & .2 \\
.7 & .3 \\
\end{array}


In [5]:
Q <- matrix(c(0.8, 0.2, 0.7,0.3), 2, 2, byrow=T)
Q

0.8,0.2
0.7,0.3


These are also intuitive to read: the  {$i^{th},j^{th}$}  entry (the value in the  $i^{th}$  row and  $j^{th}$  column) marks the probability of going from State  $i$  to State  $j$ . In this case, we named the states instead of numbering them; here, then, we associate the first row and first column with the ‘Sunny’ state, and the second row and column with the ‘Rainy’ state. For example, the probability of going from Sunny to Rainy is coded here in the first row and second column ($.2$).

It’s important to note, again, that the rows of these transition matrices must sum to $1$. That’s because from a certain state, you must go somewhere. Again, if these probabilities added up to something like $.6$, you wouldn’t know where the state traveled to $40\%$  of the time! These transition matrices will also always be square (i.e., same number of rows as columns) since we want to keep track of the probability of going from every state to every other state, and they will always have the same number of rows (and same number of columns) as the number of states in the chain.

## The Markov Property

All of this is well and good, but we still haven’t gotten to what really makes a Markov Chain Markov. Formally, a Markov Chain must have the ‘Markov Property’. This is somewhat of a subtle characteristic, and it’s important to understand before we dive deeper into Markov Chains. Take this Markov Chain, for example, where the states are labeled more generically as 1, 2 and 3.

![](MC2.png)

(Take a second to convince yourself that if you sum transition probabilities associated with the edges leading out of any particular node, you get 1.)

Now, the essence of the Markov Property is that **the future only depends on the immediate past**. That is, you only have to condition on one previous step to get all of the relevant information about predicting the future; conditioning on steps further in the past gives *no valuable information*. This property is also called *memorylessness*.

Think about this concept in the context of this specific chain. Say we arbitrarily start at node 2; as the chain is defined above, we have a $.1$ probability of going to State 1. Now imagine that we ‘run’ the chain for  $t$  periods (allow the ‘random particle’ to bounce around in the chain $t$ times), and say that each period the particle happens to take the ‘self-referencing’ loop back to State 2 (the probability of this happening would be  $.9^t$ , since each step has probability $.9$ and we use the multiplication rule, but we’re not worried about that at the moment). So, given that we’re at State 2 at time $t$, what’s the probability that we’re in State 1 at time $t+1$? Well, it’s intuitively still $.1$, just like it was the first time we started at State 2, just like, in fact, it is every single time we are in State 2.

The point is, it doesn’t matter how you got there (to State 2). If we’re thinking about the probability of transitioning to State 1, it only matters that we’re in State 2 at the previous step. We don’t have to look any further back in history, and considering the behavior of the chain further in the past will not change the probability that we transition to State 1 in the next step.



Let’s pin this down with some notation. Let $X$ be the stochastic process described by this chain (remember, a stochastic process is just a random variable that evolves through time). $X$ can clearly take on the values 1, 2 or 3 (the different possible states). Let $X_t$ be the value of the process (what state we are in) at time $t$ (after we let the ‘random particle’ move states $t$ times). Then:

$P(X_{t+1} = i | X_{t} = j) = P(X_{t+1} = 1 | X_{t} = j, ..., X_0 = x_0)$

The left hand side of this equation is giving the probability of being in State $i$ at period $t+1$, given that we were in State $j$ at period $t$. In our case above, the probability of being at State 1 at time $t+1$ given that we are in State 2 at time $t$ is $.1$. Then, the right hand side of the equation is the probability of being at State $i$  at time $t+1$ given the entire past history of the chain, or the location of the particle in every single State (here, the State $j$ at time $t$, and all the way down to $x_0$, which is just arbitrary notation for some constant that $X_0$ takes on at time $0$).



The Markov Property states that these sides are equal; that is, knowing where you were in the previous period makes the rest of the chain history irrelevant. If we know $X_t=j$, then the rest of the history doesn’t add any information for predicting $X_{t+1}$; that’s why the two sides are equal.



---
### Examples

* Imagine that you have a stochastic process that models the cumulative sum of fair, six-sided die rolls, which we’ll notate as $X$. So, you roll the die once and maybe get a $3$; this means $X_1=3$. You roll the die again and get $5$; take the sum of the two rolls and see that  $X_2=5+3=8$. 

Could we write this as a Markov Chain? That is, does this satisfy the Markov Property? Let’s think about this. Say that we are interested in the probability that $X_5$ is equal to $15$, and imagine that we get to observe that $X_4$ was equal to $12$. Well, if we know the value of $X_4$, does knowing $X_3$, $X_2$ and $X_1$ matter?

They don’t. It only matters that we got to (and are currently sitting at) the value $12$; it does not matter how we got there. The conditional probability (conditioning on $X_4=12$) that $X_5$ is $15$ is $16$, since this is the same probability as rolling a $3$ on your next roll (which means going from $12$ to $15$), regardless of how we got to $X_4=12$. So, using this kind of ‘story’ or ‘heuristic’ proof, **this process is Markovian**.



* Now consider a different stochastic process $Y$ that keeps track of the chain of letters in the book “The Adventures of Tom Sawyer’. The first word in the book is ‘TOM’, so in this case $Y_1=T$, $Y_2=O$ and $Y_3=M$. Is this process Markovian?

Consider the following transition probability (remember, a transition probability is the probability of transitioning from one specific state to another specific state). Say that we’re interested in the probability that $Y_{100}=i$. According to the Markov Property, we could look one letter back and get all of the information that we need to find the probability that $Y_{100}=i$. In this case, imagine that we observe and condition on $Y_{99}=u$.

If this process were truly Markovian, then looking further back in the past (i.e., $Y_{98}$, $Y_{97}$, etc.) would give us no more information about the probability of transitioning from a $u$ as the $99^{th}$ letter to an $i$ as the $100^{th}$ letter. However, this is not the case here. If we condition on $Y_{98}$ and see that $Y_{98}=q$, then suddenly we have a higher probability of going from $u$ to $i$ (because words like ‘quick’, ‘quite’, etc., where we have a $q$, then $u$, then $i$ are common). If we saw that $Y_{98}=h$, the probability of seeing $Y_{100}=i$ would drastically decrease (there aren’t many ‘hui’ combinations in the English language. Instead, we’d probably guess that $Y_{100}$ was a consonant!).

The point is that looking further back in time (past the previous state) is informative in this case, so **this process is not Markovian**. This time, it matters how we got there!


---
## Characteristics of Markov Chains

#### Recurrent vs. Transient (state characteristic)

When we say ‘state’ characteristic, we mean that states *within* a chain can be recurrent or transient, not entire chains. The definition of recurrence is that, given that you start at State $i$, you have probability 1 of eventually returning to State  i . A transient state is any state where this return probability is less than 1. Here’s an example of a chain with both transient and recurrent states. The edges are not labeled with probabilities, but assume each drawn edge is associated with a non-zero transition probability and that the transition matrix for this chain is valid.

![](MC3.png)

Here, State 1 is transient; if we start in State 1, we might circle back to State 1 for a while, but eventually we’ll go to states 2 and 3 and we will bounce around there for all of time, never returning to 1. States 2 and 3 are recurrent; if you start in either one, you know you’ll get back eventually.

Recurrence is relatively straightforward, but there are some subtleties. Consider this chain:

![](MC4.png)

Although it seems a little strange, both States 1 and States 2 are recurrent. You might guess that they are transient, since if we start at State 1, we never get to State 2. The key here, though, is in the definition: a state if recurrent if, given that you start at the state, you have a probability of 1 of returning. Clearly, if you start in 1 or 2, you will definitely return (over and over and over again, until the end of time!). 

#### n-Step Transition Matrix

Recall how we write our transition matrix $Q$. We say that $q_{i,j}$, or the {$i,j_{th}$}  entry in $Q$ is the transition probability of going from State $i$ to State $j$. For example:

$Q$ = \begin{array}
{rr}
.4 & .6 \\
.5 & .5 \\
\end{array}

This is a chain with 2 states (which we can label 1 and 2). The probability of going from State 1 back to State 1 is $.4$ (the entry in the first row, first column), the probability from going from State 1 to State 2 is $.6$ (first row, second column), and State 2 has equal probabilities of going back to State 2 or to State 1 (second row is  (1/2,1/2)). This transition matrix gives the probability of moving from state to state in 1-step (that is, going from State 1 to State 2 in a single step). How could we find the probability of going from state to state in multiple steps? For example, if we start at State 1, what is the probability that, after 3 steps, we are at State 2?

This seems like a complicated problem, but the answer boils down quite nicely. A proof that we won’t recreate here shows that  $Q^n$ , or the transition matrix raised to the power of $n$, gives the $n$-step transition matrix. That is, the {$i,j_{th}$}  entry of $Q^n$ gives the probability of going from State $i$ to State $j$ in exactly $n$ steps. To answer our question above (if we start at State 1, what is the probability that we are at State 2 three steps later) we would simply raise our matrix to the power of 3 and look at the entry in the first row and second column.

We can perform the above ‘matrix to a power’ calculation using the `matrix` command (the `nrow` argument determines the number of rows, and the `ncol` argument determines the number of columns). We use `%^%` to raise matrices to a power. 

In [6]:
#define the matrix
Q = matrix(c(.4, .6,
             .5, .5), nrow = 2, ncol = 2, byrow = TRUE)

#raise the matrix to the power of 3
Q%^%3

0.454,0.546
0.455,0.545


After performing this calculation, we see that entry in question is $.546$, meaning that the probability that we start in State 1, go 3 steps and end up in State 2 is $.546$. Notice that the rows of this matrix still sum to 1; that’s because this is still a valid transition matrix, just generalized to multiple steps. Also notice that the rows are starting to look similar to each other; more on this in the next section.

## Stationary Distributions

This is probably the trickiest part of our work with Markov Chains. The stationary distribution, which we notate by $s$, can (generally, usually, under certain conditions) be thought of as the long-run distribution of our chain. What does that mean? Well, if we have a Markov Chain, and we let our random particle bounce around for a very long time, then $s$ describes the distribution of the location of the particle. Even more specifically, you can think of $s_1$ as the proportion of time that the particle is in State 1 over the long run, and $s$ codes the proportion of time that the particle is in each of the states (not just State 1). We will formalize this later.

By this notation, then, if we have $m$ states in our chain, $s$ is a 1 by $m$ vector: one row by $m$ columns. The $i^{th}$ value in the vector gives the amount of time that the chain will spend in State $i$. Before even doing any math, we can guess at the stationary distribution of this chain, which we will call the ‘coin’ chain:

![](MC5.png)

As you might have guessed, this chain models the process of randomly flipping a fair, two-sided coin. The states are simply what side the coin shows on the $i^{th}$ flip. Knowing that the stationary distribution is the proportion of time spent in each state, we can guess that $s=(.5,.5)$. That is, we believe that we’re going to spend half of our time in the heads state, and half in the tails state.

This is, of course, a pretty simple example, and things get more difficult as the chains grow more complicated themselves. Now, let’s discuss more properties of the stationary distribution, and how to find it in some simple cases.

In general, the stationary distribution $s$ is given by $sQ = s$.

Where, again, $Q$ is the transition matrix. We know that $s$ is a $1\times m$ vector and $Q$ is a $m\times m$ matrix, so the multiplication on the left produces something that is $1\times m$  on the right (this makes more sense if you are familiar with linear algebra; don’t worry about it too much if you are not). Let’s step through this intuition. We saw earlier that the entries of $Q^m$ give us the probability of traveling from state-to-state in exactly $m$ steps. What about if we don’t know which state we will start in? That is, what if $X_t$ is random, and we have its distribution (that is, the distribution of where the chain will be at time $t$)? How can we find the distribution of $X_{t+1}$ (i.e., the distribution of where the chain will be at time $t+1$)?

When we consider a ‘distribution’ in this setting, we simply just mean the probability that $X_t$ or $X_{t+1}$ takes on any of the states in the matrix. We can write this as a $1\times m$ vector; a single row where the $i^{th}$ entry codes the probability of being in the $i^{th}$ state. Let’s call this vector $\theta$. In general, if $\theta$ is the distribution of where the particle lands at $X_t$, then the distribution of where the particle lands at $X_{t+1}$ is given by $\theta Q$. (Hopefully this makes intuitive sense - we multiply $Q$ by the ‘starting weights’ (i.e., distribution at the current step) to find the distribution for the next step.)

---
### Example

Consider what $\theta$ would be in the ‘coin chain’ example above for $X_1$; i.e., the distribution of where the chain starts. Since we are flipping a fair coin, we are equally likely to start with a heads or tails, meaning $\theta=(1/2,1/2)$. Therefore, if we want the distribution of $X_2$ (where the coin goes on the next move) we write $\theta Q$.

We know $\theta=(1/2,1/2)$, and we know $Q$ is a $2\times 2$ matrix full of 1/2 (going from any side of the coin to any other side of the coin has probability 1/2) so we can do this calculation in R. Similar to the power calculation above, we use `%*%` for matrix multiplication.

In [15]:
#define Q
Q = matrix(c(1/2, 1/2, 
             1/2, 1/2), ncol = 2, nrow = 2)
theta = c(1/2, 1/2)

theta%*%Q

0.5,0.5


The output comes to $(1/2,1/2)$, meaning that the distribution of $X_2$ (if we start $X_1$ at heads or tails with equal probabilities) is 1/2 heads, and 1/2 tails. No surprises here.

---
Anyways, the point here is that if $X_t$ has distribution $\theta$ and $Q$ is the transition matrix, then $\theta Q$ gives the distribution of $X_{t+1}$. Let’s return, then to the result we saw with stationary distributions: $sQ=s$. This is pretty interesting: if the distribution of $X_t$ is $s$, then this equation shows that the distribution of $X_{t+1}$ is also $s$. That is, once we get to $s$, we stay in it; the following states also all have this distribution! This is really interesting, and gives credence to the name ‘stationary.’ In fact, in the coin chain we just worked with above, when we used $(1/2,1/2)$ for $\theta$ and multiplied it by $Q$, we got $(1/2,1/2)$ back; this shows that $(1/2,1/2)$ is in fact the stationary distribution, which we guessed at earlier!

Technically speaking, you could always solve $sQ=s$ if you had $Q$, but when you have a large number of states it gets computationally difficult. 

### Long-run MC Behavior

Recall how we introduced the stationary distribution: a vector that describes the long-run behavior of the chain. Formally, if we have a Markov Chain with stationary distribution $s$, then the distribution of $X_t$ converges to $s$ as $t$ --> $\infty$  (the only condition is that $Q^n$ is positive in all entries, where $Q$ is the transition matrix and $n$ is a power that we raise it to).

More specifically, if we raise $Q$ to a large enough power (i.e., take enough steps) every row of $Q$ starts to approach the stationary distribution. Does this make sense? Well, we just saw that the MC approaches the stationary distribution, and we know $sQ=s$ (once we are in the stationary distribution, we stay there), so every row of  Q  should be $s$. Consider the first row: this is a $1\times m$  vector that codes the probability of going from State 1 to any other state. If we are in the stationary distribution, this should be $s$ (no matter what state we are coming from, since recall that once $X_t$ has the stationary distribution, so does $X_{t+1}$) so we should see this row (and all other rows) become $s$. Recall the R example from earlier, where we found the 3-step transition matrix of a $2\times2$ Markov Chain:

In [17]:
#define the matrix
Q = matrix(c(.4, .6,
             .5, .5), nrow = 2, ncol = 2, byrow = TRUE)

#raise the matrix to the power of 3
Q%^%3

0.454,0.546
0.455,0.545


Here, we can already start to see the MC approaching the stationary distribution: already, the rows are extremely similar. If we raise it to a higher power, then we see the two rows equalize:

In [21]:
#define the matrix
Q = matrix(c(.4, .6,
             .5, .5), nrow = 2, ncol = 2, byrow = TRUE)

#raise the matrix to the power of 3
Q%^%10

0.4545455,0.5454545
0.4545455,0.5454545


Here, it looks like the stationary distribution is around $s=(.4545,.5454)$ (it spends 45.45% of time in State 1, and 54.54% of time in State 2 in the long run).

In [36]:
#replicate
sims = 10000

#define the transition matrix
Q = matrix(c(.4, .6,
             .5, .5), nrow = 2, ncol = 2, byrow = TRUE)

#keep track of X (the stochastic process)
X = rep(NA, sims)

#start at a random spot
X[1] = sample(1:2, 1)

#run the loop
for(i in 2:sims){
  
  #sample according to the transition matrix;
  #   use the row associated with X_{i - 1} 
  #   for the sample probabilities
  X[i] = sample(1:2, 1, prob = Q[X[i - 1], ])
}


#should spend an equal amount of time in each state
table(X)/sims

X
     1      2 
0.4625 0.5375 

## Assignment

In [8]:
library(expm)

**Use R code introduced in lab to calculate probabilities using matrices.**

Use the following to answer questions 1-2: In the Dark Ages, Harvard, Dartmouth, and Yale admitted only male students. Assume that, at that time, 80 percent of the sons of Harvard men went to Harvard and the rest went to Yale, 40 percent of the sons of Yale men went to Yale, and the rest split evenly between Harvard and Dartmouth; and of the sons of Dartmouth men, 70 percent went to Dartmouth, 20 percent to Harvard, and 10 percent to Yale.

1. Write out the transition probability matrix. Find the probability that the grandson of a man from Harvard went to Harvard. 

In [ ]:
q1 <- NULL

2. Modify the above by assuming that the son of a Harvard man always went to Harvard. Again, find the probability that the grandson of a man from Harvard went to Harvard.

In [ ]:
q2 <- NULL

3. Write down the transition probabilities of the Markov chain (Hint: recall Punnett squares! If you don't remember, a quick Google image search should do the trick).

In [2]:
q3 <- NULL

4. Assume that we start with a hybrid rabbit. Let $\mu n$ be the probability distribution of the character of the rabbit of the $n^{th}$ generation. In other words, $\mu n$(GG), $\mu n$(Gg), $\mu n$(gg) are the probabilities that the $n^{th}$ generation rabbit is GG, Gg, or gg, respectively. Compute $\mu_2$, $\mu_3$, $\mu_4$. What do you notice about $\mu n$ in general? (Create as many new variables or cells you want in order to show each $n^{th}$-generation matrix.)

In [ ]:
q4 <- NULL

[**Fun explanation for #2 once you've solved it**:

Actually the probabilities are the same for any $i \in N$. If you obtained this result before 1858 when Gregor Mendel started to breed garden peas in his monastery garden and analysed the offspring of these matings, you would probably be very famous because it definitely looks like a law! This is what Mendel found when he crossed mono-hybrids. In a more general setting, this law is known as Hardy-Weinberg law.]

Use the following to answer questions 5-6: A certain calculating machine uses only the digits 0 and 1. It is supposed to transmit one of these digits through several stages. However, at every stage, there is a probability $p$ that the digit that enters this stage will be changed when it leaves and a probability $q = 1 − p$ that it won't. Form a Markov chain to represent the process of transmission by taking as states the digits 0 and 1.


5. What is the matrix of transition probabilities?

In [4]:
q5 <- NULL

6. Now draw a Markov chain diagram and assign probabilities assuming that the process begins in state 0 and moves through two stages of transmission (do this for yourself on a sheet of paper - no need to submit the diagram). What is the probability that the machine, after two stages, produces the digit 0 (i.e., the correct digit)? (This question doesn't need any R code - submit the answer below as an expression.)

In [ ]:
q6 <- NULL